In [ ]:
from benchmark import Benchmark, FixedIterator

bm = Benchmark('FPM', 'N', [2**i for i in range(6, 13)], t_min=0.5)

In [ ]:
# Physical parameters
diameter = 20e-3
pupil_diameter = 10e-3  # m
wavelength = 632e-9  # m
F_number = 80
fpm_diameter = 300e-6  # m

focal_length = F_number * pupil_diameter
resolution_element = wavelength * F_number

# Numerical parameters
N = 512
q_fpm = 64

In [ ]:
import hcipy

def fpm_hcipy(it, N):
    pupil_grid = hcipy.make_pupil_grid(N, diameter)

    fpm_grid = hcipy.make_focal_grid(q_fpm, fpm_diameter / resolution_element / 2, resolution_element)
    fpm = 1 - hcipy.make_circular_aperture(fpm_diameter)(fpm_grid)

    coro = hcipy.LyotCoronagraph(pupil_grid, fpm, focal_length=focal_length)

    aperture = hcipy.make_circular_aperture(pupil_diameter)(pupil_grid)
    wf = hcipy.Wavefront(aperture, wavelength)

    for _ in it:
        lyot = coro(wf)

    return lyot.electric_field

bm.run(fpm_hcipy, 'hcipy')

hcipy.imshow_field(fpm_hcipy(FixedIterator(), 512))

In [ ]:
from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import circle
from prysm.propagation import Wavefront

def fpm_prysm(it, N):
    xi, eta = make_xy_grid(N, diameter=diameter * 1e3)

    r, t = cart_to_polar(xi, eta)
    A = circle(pupil_diameter * 1e3 / 2, r)

    dx = xi[0, 1] - xi[0, 0]

    wf = Wavefront.from_amp_and_phase(A, None, wavelength * 1e6, dx)

    N_fpm = q_fpm * fpm_diameter / resolution_element

    xi_fpm, eta_fpm = make_xy_grid(N_fpm, diameter=diameter * 1e3)
    dx_fpm = xi_fpm[0, 1] - xi_fpm[0, 0]

    r_fpm, t = cart_to_polar(xi_fpm, eta_fpm)

    fpm = 1 - circle(fpm_diameter * 1e3 / 2, r_fpm)

    for _ in it:
        E = wf.to_fpm_and_back(focal_length * 1e3, 1 - fpm, dx_fpm)

    return E.data

bm.run(fpm_prysm, 'prysm')

In [ ]:
bm.plot()